In [ ]:
import os
import numpy as np
import tensorflow as tf
import time
from datetime import timedelta
from temporalcontext import settings
from temporalcontext.functions import read_selmap, read_folds_info


# Given the use LoG operator for input conditioning, it's better start
# with a lower learning rate.
def variable_learning_rate(epoch):
    var_rates = [0.001 * factor for factor in [1.0, 1e-1, 1e-2, 1e-3]]
    for idx, boundary in enumerate([10, 35, 45]):
        if epoch <= boundary:
            return var_rates[idx]
    return var_rates[-1]


In [ ]:
selmap = read_selmap(os.path.join(settings.raw_data_root, 'selmap.csv'))
fold_file_idxs = read_folds_info(os.path.join(settings.raw_data_root, 'folds_info.txt'))

# Loop over each unique segment advance
for seg_adv, ts in list(set([(ex['segment_advance'], ex['time_steps']) for ex in settings.lstm_experiments])):
    
    random_state = np.random.RandomState(settings.random_seed)

    segments_root = os.path.join(settings.project_root,
                                 settings.segments_dir,
                                 'seg_adv_{:.2f}'.format(seg_adv))
    
    train_times = list()
    
    for fold_idx, fold_info in enumerate(fold_file_idxs):
        print('===== Fold {:02d}, segment advance {:.2f} s ====='.format(fold_idx + 1, seg_adv))

        # Load all positive and negative training samples for the fold
        pos_samples = list()
        neg_samples = list()
        for f_idx in fold_info['train']:
            au_file = selmap[f_idx][0]
            
            with np.load(os.path.join(segments_root, settings.class_dirs[0], au_file + '.npz'), 'r') as data:
                pos_samples.append(data['segments'])
                
            with np.load(os.path.join(segments_root, settings.class_dirs[1], au_file + '.npz'), 'r') as data:
                neg_samples.append(data['segments'])
        
        pos_samples = np.concatenate(pos_samples, axis=0)
        neg_samples = np.concatenate(neg_samples, axis=0)
        
        # Shuffle the indices for random splitting into train & eval subsets.
        # Then, restrict to limits.
        pos_samples_idxs = random_state.permutation(pos_samples.shape[0])[:settings.max_per_class_training_samples]
        neg_samples_idxs = random_state.permutation(neg_samples.shape[0])[:settings.max_per_class_training_samples]
        
        num_pos_samples = len(pos_samples_idxs)
        num_neg_samples = len(neg_samples_idxs)
        total_samples = num_pos_samples + num_neg_samples

        # Identify the points where each class' samples are to be split
        pos_eval_split = int(round((1.0 - settings.validation_split) * num_pos_samples))
        neg_eval_split = int(round((1.0 - settings.validation_split) * num_neg_samples))
        
        num_train_samples = pos_eval_split + neg_eval_split
        
        class_weights = {
            0: num_train_samples / (2 * neg_eval_split),
            1: num_train_samples / (2 * pos_eval_split)
        }

        # Combine train/eval pos & neg samples splits
        train_data_x = np.concatenate([pos_samples[pos_samples_idxs[:pos_eval_split], ...],
                                       neg_samples[neg_samples_idxs[:neg_eval_split], ...]], axis=0)
        train_data_y = np.concatenate([np.repeat([[0, 1]], pos_eval_split, axis=0),
                                       np.repeat([[1, 0]], neg_eval_split, axis=0)], axis=0)
        eval_data_x = np.concatenate([pos_samples[pos_samples_idxs[pos_eval_split:], ...],
                                      neg_samples[neg_samples_idxs[neg_eval_split:], ...]], axis=0)
        eval_data_y = np.concatenate([np.repeat([[0, 1]], num_pos_samples - pos_eval_split, axis=0),
                                      np.repeat([[1, 0]], num_neg_samples - neg_eval_split, axis=0)], axis=0)
        
        # Add channel dimension
        train_data_x = np.expand_dims(train_data_x, axis=3)
        eval_data_x = np.expand_dims(eval_data_x, axis=3)
        
        print('Positive samples : {:7d} training, {:5d} eval'.format(
            pos_eval_split, num_pos_samples - pos_eval_split))
        print('Negative samples : {:7d} training, {:5d} eval'.format(
            neg_eval_split, num_neg_samples - neg_eval_split))
        print('Total samples    : {:7d} training, {:5d} eval'.format(
            train_data_x.shape[0], eval_data_x.shape[0]))
        
        val_steps = np.ceil(eval_data_x.shape[0] / settings.batch_size).astype(np.int)
        
        # Shuffle training group so that the classes aren't lumped together
        shuffle_idxs = random_state.permutation(train_data_x.shape[0])
        
        tf.keras.backend.clear_session()
        tf.random.set_seed(settings.random_seed)
        
        train_dataset = tf.data.Dataset.from_tensor_slices((train_data_x[shuffle_idxs, ...],
                                                            train_data_y[shuffle_idxs, ...]))
        train_dataset = train_dataset.cache().shuffle(settings.buffer_size).batch(settings.batch_size)

        eval_dataset = tf.data.Dataset.from_tensor_slices((eval_data_x, eval_data_y))
        eval_dataset = eval_dataset.batch(settings.batch_size).repeat()
        
        # Load the model architecture
        with open('baseCNN_architecture.json', 'r') as json_file:
            classifier = tf.keras.models.model_from_json(json_file.read())
        
        #classifier.summary()
        
        classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=variable_learning_rate(0)),
                           loss=tf.keras.losses.CategoricalCrossentropy(),
                           metrics=[tf.keras.metrics.CategoricalAccuracy()])

        # Train
        start_time = time.time()
        history = classifier.fit(
            x=train_dataset,
            epochs=settings.epochs,
            validation_data=eval_dataset,
            validation_freq=settings.epochs_between_evals,
            validation_steps=val_steps,
            class_weight=class_weights,
            initial_epoch=0,
            shuffle=False,
            callbacks=[tf.keras.callbacks.LearningRateScheduler(variable_learning_rate)],
            verbose=2)
        end_time = time.time()

        train_times.append(end_time - start_time)
        
        # Save trained model. Reset metrics before saving
        classifier.reset_metrics()
        model_dir = os.path.join(settings.project_root, settings.folds_dir,
                                 'f{:02d}'.format(fold_idx + 1), 'seg_adv_{:.2f}'.format(seg_adv),
                                 settings.models_dir)
        os.makedirs(model_dir, exist_ok=True)
        classifier.save(os.path.join(model_dir, 'baseCNN.h5'))

        # Free up some memory before next iteration
        del classifier, train_dataset, eval_dataset, train_data_x, train_data_y, eval_data_x, eval_data_y
        
    print('----------------------------------------')
    print('Training times for segment advance {:.2f} s:'.format(seg_adv))
    print('    Min = {}'.format(timedelta(seconds=min(train_times))))
    print('    Max = {}'.format(timedelta(seconds=max(train_times))))
    print('    Avg = {}'.format(timedelta(seconds=sum(train_times) / len(train_times))))
    print('========================================')
    print()
